<a href="https://colab.research.google.com/github/Crisqin/RL_FrozenLake8x8-v0/blob/main/FrozenLake8x8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# using genetic algorithm to solve open ai gym solution

In [2]:
import numpy as np
import gym
env = gym.make('FrozenLake8x8-v0')
env.seed(0)

def evaluate_policy(env, policy):
  total_rewards = 0.0
  for _ in range(100):
    obs = env.reset()
    while True:
      action = policy[obs]
      obs, reward, done, info = env.step(action)
      total_rewards += reward
      if done:
        break
  return total_rewards/100

def crossover(policy1, policy2):
  new_policy = policy1.copy()
  for i in range(16):
      rand = np.random.uniform()
      if rand > 0.5:
          new_policy[i] = policy2[i]
  return new_policy

def mutation(policy):
  new_policy = policy.copy()
  for i in range(64):
    rand = np.random.uniform()
    if rand < 0.05:
      new_policy[i] = np.random.choice(4)
  return new_policy

k=25
policy_pop = [np.random.choice(4, size=((64))) for _ in range(100)]
for idx in range(25):
  policy_scores = [evaluate_policy(env, pp) for pp in policy_pop]
  policy_ranks = list(reversed(np.argsort(policy_scores)))
  elite_set= [policy_pop[x] for x in policy_ranks[:k]]
  select_probs = np.array(policy_scores) / np.sum(policy_scores)
  child_set = [crossover(
      policy_pop[np.random.choice(range(100), p=select_probs)], 
      policy_pop[np.random.choice(range(100), p=select_probs)])
      for _ in range(100 - k)]
  k-=1
  mutated_list = [mutation(c) for c in child_set]
  policy_pop = elite_set
  policy_pop += mutated_list
policy_score = [evaluate_policy(env, pp) for pp in policy_pop]
best_policy = policy_pop[np.argmax(policy_score)]
print('Best actions score =', (np.max(policy_score)),'best actions =', best_policy.reshape(8,8))
env.close()

Best actions score = 0.93 best actions = [[2 2 3 2 1 1 2 2]
 [3 3 3 3 3 3 2 2]
 [2 3 3 0 2 1 2 2]
 [3 1 1 2 0 1 2 2]
 [2 0 3 1 1 1 3 2]
 [1 2 2 0 1 0 3 2]
 [3 2 0 0 3 3 0 2]
 [1 3 2 0 0 0 2 0]]


# policy_evaluation

In [2]:
import gym
import time
import numpy as np

def policy_evaluation(env, value_table, policy, gamma=0.9, threshold=1e-4):
    delta = 2 * threshold
    while delta > threshold:
        # 1.存储旧的value表
        new_value_table = np.zeros(env.nS)  # 此处不能用np.copy(value_table),因为更新V(s)用的是+=，所以若不置零则无限加和不收敛
        for state in range(env.nS):
            # 从当前state提取策略对应的action
            action = policy[state]          # 可能会由于随机选择的动作不包含奖励=1的动作而得到无更新的new_value_table
            # 2.更新V(s)
            for prob, next_state, reward, done in env.P[state][action]:
                new_value_table[state] += prob * (reward + gamma*value_table[next_state])
        delta = sum(np.fabs(new_value_table - value_table))
        value_table = new_value_table
    return value_table

def policy_improvement(env, value_table, policy, gamma=0.9):
    while True:
        # 1.存储旧policy
        old_policy = np.copy(policy)
        for state in range(env.nS):
            action_value = np.zeros(env.nA)
            for action in range(env.nA):
                for prob, next_state, reward, done in env.P[state][action]:
                    action_value[action] += prob * (reward + gamma*value_table[next_state])
            # 2.更新最优policy
            policy[state] = np.argmax(action_value)
        if np.all(policy == old_policy):  break
    return policy
    
def policy_iteration(env, iterations, gamma=0.9):
    env.reset()
    start = time.time()
    # 初始化策略-随机策略 (16个状态下的[0,4)策略)
    policy = np.random.randint(low=0, high=env.nA, size=env.nS)
    #policy = np.ones(env.observation_space.n)
    # 初始化value表 (初始化0)
    value_table = np.zeros(env.nS)
    for step in range(iterations):
        old_policy = np.copy(policy)
        # 1.Policy Evaluation
        value_table = policy_evaluation(env, value_table, policy, gamma)
        # 2.Policy Improvement
        policy = policy_improvement(env, value_table, policy, gamma)
        # 3.判断终止条件
        if np.all(policy == old_policy):
            print('===== Policy Iteration ======\nTime Consumption: {}s\nIteration: {} steps\nOptimal Policy(gamma={}): {}'.format(time.time()-start, step+1, gamma, policy))
            break
    return value_table, policy

def play_game(env, policy, episodes=5, timesteps=150):
    for episode in range(episodes):
        state = env.reset()
        for t in range(timesteps):
            action = policy[state]
            state, reward, done, info = env.step(action)
            if done:
                print("===== Episode {} finished ====== \n[Reward]: {} [Iteration]: {} steps".format(episode+1, reward, t+1))
                env.render()
                break

# 创建冰湖环境
env = gym.make('FrozenLake8x8-v0')
# 策略迭代
for i in np.arange(0,1,0.1):
  value_table, policy = policy_iteration(env, iterations=100000, gamma=i)
  # 使用迭代计算得到的策略打游戏
  play_game(env, policy, episodes=3)
env.close()

===== Policy Iteration ======
Time Consumption: 0.004427433013916016s
Iteration: 2 steps
Optimal Policy(gamma=0.0): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
===== Policy Iteration ======
Time Consumption: 0.049445390701293945s
Iteration: 11 steps
Optimal Policy(gamma=0.1): [1. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 3. 2. 1. 1. 1. 1. 2. 0. 0. 2. 3. 2. 1.
 3. 2. 3. 1. 0. 0. 2. 1. 3. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 1. 1. 0. 0. 1. 1. 1. 0.]
===== Episode 1 finished ====== 
[Reward]: 1.0 [Iteration]: 125 steps


TypeError: ignored

# value_iteration

In [4]:
import gym
import time
import numpy as np

def value_iteration(env, threshold=1e-4, gamma=0.9):
    env.reset()
    start = time.time()
    # 初始化策略
    policy = np.zeros(env.nS, dtype=int)   # 默认为float类型
    # 初始化value表 (初始化0)
    value_table = np.zeros(env.nS)
    new_value_table = np.zeros(env.nS)
    delta = 2 * threshold
    while delta > threshold:
        for state in range(env.nS):
            action_value = np.zeros(env.nA)
            for action in range(env.nA):
                for prob, next_state, reward, done in env.P[state][action]:
                    action_value[action] += prob * (reward + gamma*value_table[next_state])
            # 1.利用max操作更新V(s)，区别与Policy Iteration
            new_value_table[state] = max(action_value)
            # 2.Policy Improvement
            policy[state] = np.argmax(action_value)
        delta = sum( np.fabs(new_value_table - value_table) )
        value_table = np.copy(new_value_table)   # 注：需用copy拷贝副本，否则两个变量指向同一位置，则赋值时同时改变
    print('===== Value Iteration ======\nTime Consumption: {}s\nIteration: {} steps\nOptimal Policy(gamma={}): {}'.format(time.time()-start, 1, gamma, policy))
    return value_table, policy

def play_game(env, policy, episodes=5, timesteps=150):
    for episode in range(episodes):
        state = env.reset()
        for t in range(timesteps):
            action = policy[state]
            state, reward, done, info = env.step(action)
            if done:
                print("===== Episode {} finished ====== \n[Reward]: {} [Iteration]: {} steps".format(episode+1, reward, t+1))
                env.render()
                break

env = gym.make('FrozenLake8x8-v0')
# 价值迭代
for i in np.arange(0,1,0.1):
  value_table, policy = value_iteration(env, gamma=i)
  # 使用迭代计算得到的策略打游戏
  play_game(env, policy, episodes=3)
env.close()

===== Value Iteration ======
Time Consumption: 0.002713918685913086s
Iteration: 1 steps
Optimal Policy(gamma=0.0): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
===== Value Iteration ======
Time Consumption: 0.006482839584350586s
Iteration: 1 steps
Optimal Policy(gamma=0.1): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 1 1 1 0 0 0 0 1 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 1 1 1 0]
===== Value Iteration ======
Time Consumption: 0.007903575897216797s
Iteration: 1 steps
Optimal Policy(gamma=0.2): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1
 1 3 2 0 0 0 1 1 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 1 1 1 0]
===== Value Iteration ======
Time Consumption: 0.010517120361328125s
Iteration: 1 steps
Optimal Policy(gamma=0.30000000000000004): [0 0 0 0 0 0 1 2 0 0 0 0 0 1 1 1 0 0 0 0 1 1 2 1 0 0 0 1 0 0 2 1 0 0 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 0 0 0 1 1 1 0]
